In [45]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [46]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWI2M2NjMjgtY2Q0OC00OGRiLWFlMzMtMGNkMGFhM2ZhODFkIiwgInJlY2lwaWVudEtleXMiOiBbIkhRcEpFaTM5Yk5NRDQ5NXBRVXVSQ0ozdEprVjhZQXNXUFA5eG9Ga3hBY0JtIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [47]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYjk2YzI4YjgtNmEzMS00YjgxLTk2NzktOTljNjViNDMzMWE5IiwgImxhYmVsIjogIkNsaWVudCAoYWRtaW4pIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyIzS1Iya01rNHNMOE04eFNmRk5FWFR1Z3lhOW9ZQ3BUSlZrQXRvSGt6eksySCJdfQ==


In [48]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': 'b96c28b8-6a31-4b81-9679-99c65b4331a9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['3KR2kMk4sL8M8xSfFNEXTugya9oYCpTJVkAtoHkzzK2H'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [49]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 3onGHCb1MqHFMWrVSdX1WQTBdjqkUrSwJKc4HAhgSWkwSZnyw1fGEn8XE47zkRPsmpGrKhmvdCobWwsM12T41aTW

Public key: 
 CAF218zMaQ9Uzx4VQvfFJiKLyFd1MxSU2hui4d6YZpUC


**Sending a connection request to acapy**

In [50]:
   
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [51]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [52]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:E79ZxGB8oudGQB8k1j8Xjt#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:E79ZxGB8oudGQB8k1j8Xjt',
             'publicKey': [{'controller': 'did:sov:E79ZxGB8oudGQB8k1j8Xjt',
                            'id': 'did:sov:E79ZxGB8oudGQB8k1j8Xjt#1',
                            'publicKeyBase58': '89JmZBcrZemBYshQeGVM5GNUYv9Snp2uY1yi6cDniNUz',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:E79ZxGB8oudGQB8k1j8Xjt;indy',
                          'priority': 0,
                          'recipientKeys': ['89JmZBcrZemBYshQeGVM5GNUYv9Snp2uY1yi6cDniNUz'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'\x8c`*\n\xb9}\x0b\x992\xf9\xc8,\xf8\x7f\xb51\xfa\xa1\x88

### FEATURE DISCOVERY

In [53]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)

{'@id': 'e953c88a-0ed9-4922-8ca9-470582b37bc9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0'},
               {'pid': 'https://didcomm.org/action-menu/1.0'},
               {'pid': 'https://didcomm.org/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0'}

In [ ]:
message = util.buildMessage(
    'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/add',
    payload="{aaaaa}: bbb"
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)


In [54]:
message = util.buildMessage(
    'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/create',
    label="Agent1ToAgent2",
    alias="Agent1ToAgent2",
    auto_accept=True 
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)

{'@id': 'f8f37724-bd0e-454e-ac92-4a554047d0e6',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0/problem-report',
 'explain-ltxt': 'Unrecognized message type '
                 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/create',
 '~thread': {'thid': '8837eb8d-05f6-4c59-8ecc-3fcf8026c72d'}}


In [55]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/receive-invitation",
    accept='none',
    invitation=response['invitation_url'],
)
responsee = util.sendMessage(message, connections[1])
pp.pprint(responsee)


KeyError: 'invitation_url'

In [ ]:
message = util.buildMessage(
    "https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/get-list",
)
responseee = util.sendMessage(message, connections[1])
pp.pprint(responseee['connections'])

In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
)
list_dids = util.sendMessage(message, connections[0])
pp.pprint(list_dids)

In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)


In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schema-exchange/1.0/send",
    connection_id="d352f12e-cb92-4046-859a-4f05f4e6b40e",
    content="Test"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)


In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schema-exchange/1.0/get"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)
# "6c6642dc-0372-4414-b5e8-98e5f4a58075"